Importing all relevant packages and importing dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import zscore
from scipy.stats import skew, kurtosis
from sklearn.svm import SVC

In [ ]:
df=pd.read_csv('')

DataFrame Checks

df

In [ ]:
df.info

In [ ]:
df.describe

In [ ]:
df.isnull().sum()

In [ ]:
# Check for duplicate rows, excluding the 'id' column as 'id' column is a unique identifier for each recipe
duplicate_rows = df.duplicated(subset=df.columns.difference(['id'])) 

In [ ]:
df.head()

In [ ]:
df.tail()

Data Cleaning

Data cleaning is a critical step in the data preprocessing phase. It involves handling missing values, removing duplicates, and addressing inconsistencies or errors present in the dataset. By ensuring the cleanliness and reliability of the data, we significantly reduce the likelihood of biased or inaccurate analysis.
Data cleaning plays a pivotal role in achieving "tidy data," which is essential for conducting further operations and analyses effectively. In this section, we will meticulously analyze the data types of variables and correct them if necessary. Additionally, we will identify and handle null values in the dataset.
By adhering to robust data cleaning practices, we ensure that the dataset is in optimal condition for subsequent analysis and modeling.

In [ ]:
# Use the copy() function to make a copy of the DataFrame
df_test = df.copy()

In [ ]:
# Obtain descriptive statistics by Region using groupby()
df_test.groupby('Region').describe()

In [ ]:
# Fill in missing Region data with "Other" and compare statistics
df_test['Region'] = df_test['Region'].fillna('Other')

In [ ]:
df_test.groupby('Region').describe()

In [ ]:
df_test_2 = df.copy()

Outliers

Now that we have cleaned up the structural issues in our DataFrame, we should check for outliers. The simple way to do this is by using a BoxPlot. We can use the seaborn boxplot() function to identify outliers.

There are more sophisticated techniques, for example calculating the z-score of each value, however since we have not yet done this in class, we will use a simple approach.

In [ ]:
sns.boxplot(data = df, y='Price')

In [ ]:
# Find index values of observations above 1000
drop_price_outliers = df[df['Price']>1000].index
drop_price_outliers

In [ ]:
df.drop(drop_price_outliers, inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

We are now finished the data cleaning steps! Our final DataFrame has 890 entries we can use for analysis. To summarize how we approached cleaning the data:

We dropped rows that had multiple missing values, which would cause challenges for imputation
We reviewed missing values in detail, and compared various approaches to imputing data. For the Region, we assessed whether we should impute with "Other" or the mode - since the mode resulted in value changes, we decided to go with "Other"
For the Product, we compared the approach of "Other", the mode, or imputing with random product assignments. Neither approach was great, and since we had only 10% of missing data points we decided that as long as we fill in missing Price and Units Sold data, we can drop the missing data from this column
For Price, we compared imputation by the column median, median by Product, or median by Region. Option 1 and 3 yielded the same results, and Option 2 created bias - as such, we decided to use the column median
For Units Sold, we compared imputation by the column median, median by Product, or median by Region. All 3 options yielded similar results, so for simplicity we used the column median
We changed the data type of the Sale Date column to datetime (this was an optional step for this assignment)
We used a boxplot to identify outliers in the Price and Units Sold column, and removed outliers
We dropped all duplicate entries
Now, let's move on to Data Exploration

Data Exploration

The purpose of the data exploration stage is to identify trends and patterns in data. First, we should explore each variable by itself, and then we should explore variables in pairs or combinations. Our objective for the Data Exploration stage is to identify clear patterns or recommendations we can make to the business.

Let's start with univariate analysis.

For the continuous variables (Price and Units Sold), we can use histograms to understand spread. For categorical variables, we can use counts and bar charts to see which groups are most popular.

In [ ]:
sns.histplot(df['Price'])

In [ ]:
from scipy import stats
print(stats.skew(df['Price']))

In [ ]:
sns.histplot(df['Units Sold'])

Z-score

In [ ]:
#based on skewness and kurtosis, we use z score to remove outliers
if abs(s) > 1:
        df[column] = df[column].apply(lambda x: np.log(x + 1))

In [ ]:
z_scores = zscore(df[numerical_columns])

In [ ]:
threshold = 3

In [ ]:
df = df[(np.abs(z_scores) < threshold).all(axis=1)]

In [ ]:
# After we removing the outlier, we can see that the value of skewness and Kurtosis had changed significantly.
for column in numerical_columns:
    skewness_values = []
    s = skew(df[column])
    k = kurtosis(df[column])
    print(f'{column}: skewness = {s}, kurtosis = {k}')

In [ ]:
df.groupby('Region')['Units Sold'].count().sort_values(ascending=False).plot(kind='bar', figsize=(10,5))

In [ ]:
units_by_region['sum'].sort_values(ascending=False).plot(kind='bar')

In [ ]:
sns.lmplot(data = df, x='Price', y='Units Sold')

Feature engineering

Generate new features from the data you have. This could include:
binning (i.e., translate a continuous variable into groups like 5-10, 15-20, etc.)
dummy variables (use one hot encoding, or pandas get_dummies() function to convert categorical variables to dummies)
define new metrics (e.g., multiply columns together or create custom categories based on multiple variables)
Check the relationship of engineered features to the output variable, using the methods outlined above

Formula: Precision = True Positives / (True Positives + False Positives)
When to use precision:
When the cost of false positives is high. For example, in medical diagnoses, you want to be very certain before classifying someone as having a disease, as a false positive could lead to unnecessary treatments or stress.
When you want to prioritize the accuracy of positive predictions over other metrics.

Formula: Recall = True Positives / (True Positives + False Negatives)
When to use recall:
When the cost of false negatives is high. For instance, in spam email detection, it's important to avoid missing spam emails even if it means some legitimate emails are classified as spam (false positives).
When the goal is to capture as many positive instances as possible, even if it results in more false positives.